In [ ]:
# task 2

# 1, open a text file
# 2, for loop to get each line 
        # run decoding model Beam search
        # run decoding model Sampling


In [8]:
import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, infer_device
import pandas

# 可以将其理解为：
#     AutoTokenizer 有一堆预先定义好的神经网络结构定义
#     输入的‘google/flan-t5-base’是一个已经训练好的神经网络的参数表
#     把参数表给进AutoTokenizer，AutoTokenizer会自动识别并选择一个匹配的网络定义，整合成一个可以使用的神经网络用于下一步处理


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")



In [6]:
def beamSearchGenerator(inputs):
    return model.generate(**inputs, max_new_tokens=50, num_beams=2)

def sampleGenerator(inputs):
    return model.generate(**inputs, do_sample=True, num_beams=1)

In [19]:
file_path = "./article.txt"

data = {
    "input": [],
    'output': [],
    'decoding':[],
    'tokens_out':[],
    'constraint_passed':[],
    'notes':[]
}

df = pandas.DataFrame(data)

def count(input):
    return input['input_ids'].size(dim=1)

def validator(input):
    if not input[0].endswith("."):
        return 'No period'
    if len(input[0]) < 10:
        return 'Too short'
    return True

sum_passed_constraints_bs = 0
sum_passed_constraints_s = 0
avg_token = 0
total = 0

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        line = line.rstrip("\n")  # remove trailing newline
        if len(line) > 0:
            
            total += 1
            
            inputs = tokenizer(line, return_tensors="pt")
            
            outputs_bs = beamSearchGenerator(inputs)
            outputs_s = sampleGenerator(inputs)
            
            generate_bs = tokenizer.batch_decode(outputs_bs, skip_special_tokens=True)
            generate_s = tokenizer.batch_decode(outputs_s, skip_special_tokens=True)
            
            # print("Line: " + line)
            # print(inputs)
            # print(outputs_bs)
            # print(generate_bs)
            # print(outputs_s)
            # print(generate_s)
            
            value = count(inputs)
            
            avg_token += value
            
            validation_bs = validator(generate_bs)
            validation_s = validator(generate_s)
            
            validation_pass_bs = True if validation_bs == True else False
            validation_pass_s = True if validation_s == True else False
            
            if validation_pass_bs == True:
                sum_passed_constraints_bs += 1
            if validation_pass_s == True:
                sum_passed_constraints_s += 1
            
            notes_bs = "" if validation_bs == True else validation_bs
            notes_s = "" if validation_s == True else validation_s
            
            df.loc[len(df)] = [
                line, generate_bs[0], 'beam_search', value, validation_pass_bs, notes_bs
            ]
            
            df.loc[len(df)] = [
                line, generate_s[0], 'sampling', value, validation_pass_s, notes_s
            ]
            
            
df.to_csv('results.csv')

avg_token /= total

print('% of rows paased constrains (bs): ', sum_passed_constraints_bs / total)
print('% of rows paased constrains (s): ', sum_passed_constraints_s / total)
print('average tokens out: ', avg_token)
print('total: ', total)


% of rows paased constrains (bs):  0.9130434782608695
% of rows paased constrains (s):  0.13043478260869565
average tokens out:  37.78260869565217
total:  23
